# Here we are going to run some simulations for Red Thread matchmaking!!
If all goes well I may even come up with some innovative formulas :3

## First we gotta setup some python stuff
Run the following commands to get python setup correctly

In [ ]:
!pip3 install numpy matplotlib geopandas contextily rtree geodatasets

### Now that we have installed these libraries let's go ahead and import them. we will be using them a lot!!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Key considerations
The following probability distributions will be key to matchmaking

### User age

First let's define some key parameters. When we get real data we can extract these key paramters from real data!!!! For now we have to be creative

In [ ]:
# Parameters for the normal distribution
mean_age = 30  # Average age
std_dev_age = 5  # Standard deviation
num_of_users = 10000  # Number of users

Let's use these parameters to generate some data. We also need to impose the requirement that all users be at least 18 years old.

In [ ]:
# Generate a large amount of data to ensure we have enough valid ages
initial_sample_size = num_of_users * 2  # Generate twice the required number of users
age_data = np.random.normal(mean_age, std_dev_age, initial_sample_size)

# Filter out ages below 18
age_data = age_data[age_data >= 18]

# If we have more than enough valid ages, we take only the first num_of_users
if len(age_data) > num_of_users:
    age_data = age_data[:num_of_users]
else:
    # If not enough, we generate more data until we have enough valid ages
    while len(age_data) < num_of_users:
        additional_data = np.random.normal(mean_age, std_dev_age, num_of_users)
        additional_data = additional_data[additional_data >= 18]
        age_data = np.concatenate((age_data, additional_data))

        # Keep only the required number of users
        if len(age_data) > num_of_users:
            age_data = age_data[:num_of_users]

Now that we have a good dataset let's plot it

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(age_data, bins=30, edgecolor='black', alpha=0.7)
plt.title('User Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

### Gender Distribution

In [ ]:
# Gender Distribution
gender_distribution = [0.7, 0.3]  # 70% male, 30% female

# Generate gender data
genders = np.random.choice(['Male', 'Female'], size=num_of_users, p=gender_distribution)

# Count occurrences of each gender
unique, counts = np.unique(genders, return_counts=True)
gender_counts = dict(zip(unique, counts))

# Plotting the gender distribution
plt.figure(figsize=(8, 8))
plt.pie(gender_counts.values(), labels=gender_counts.keys(), autopct='%1.1f%%', startangle=140)
plt.title('User Gender Distribution')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

## User Location

This is where it gets hard. Let's assume that everyone is in NYC for now

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx
from rtree import index
from geodatasets import get_path

# Load the built-in NYC borough boundaries dataset
nyc_land = gpd.read_file(get_path('nybb'))
nyc_land = nyc_land.to_crs(epsg=4326)  # Ensure it is in WGS84 coordinate system

# Population densities for NYC boroughs (in people per square mile)
# Source: Approximate data from various sources
population_densities = {
    'Manhattan': 72000,
    'Bronx': 33000,
    'Brooklyn': 38000,
    'Queens': 21000,
    'Staten Island': 8300
}

# Assign probabilities based on relative population densities
total_density = sum(population_densities.values())
borough_probs = {k: v / total_density for k, v in population_densities.items()}

# Parameters
num_of_users = 1000

# Function to generate random points within a borough
def generate_random_points_within_borough(polygon, num_points):
    points = []
    minx, miny, maxx, maxy = polygon.bounds

    while len(points) < num_points:
        random_points = [Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy)) for _ in range(num_points)]
        for point in random_points:
            if polygon.contains(point):
                points.append(point)
                if len(points) >= num_points:
                    break
    return points

# Generate random points for each borough according to their population density probabilities
random_points = []
for borough, prob in borough_probs.items():
    borough_polygon = nyc_land[nyc_land['BoroName'] == borough].geometry.unary_union
    num_points = int(prob * num_of_users)
    random_points.extend(generate_random_points_within_borough(borough_polygon, num_points))

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame({'geometry': random_points})
gdf.crs = 'EPSG:4326'

# Plotting the user locations
fig, ax = plt.subplots(figsize=(10, 10))
nyc_land.boundary.plot(ax=ax, linewidth=1)
gdf.plot(ax=ax, markersize=5, color='blue', alpha=0.6)
ctx.add_basemap(ax, crs='EPSG:4326', source=ctx.providers.CartoDB.Positron)
plt.title('User Geographical Distribution in New York City')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()


## User BMI

In [ ]:
# Parameters for the normal distribution of BMI
mean_bmi_men = 26  # Average BMI for men
std_dev_bmi_men = 4  # Standard deviation for men
mean_bmi_women = 24  # Average BMI for women
std_dev_bmi_women = 4  # Standard deviation for women

# Function to generate valid BMI values
def generate_valid_bmi(mean, std, num_users):
    bmi_data = np.random.normal(mean, std, num_users * 2)  # Generate twice the number of users initially
    bmi_data = bmi_data[(bmi_data >= 15) & (bmi_data <= 40)]  # Filter out unrealistic BMI values

    while len(bmi_data) < num_users:
        additional_data = np.random.normal(mean, std, num_users)
        additional_data = additional_data[(additional_data >= 15) & (additional_data <= 40)]
        bmi_data = np.concatenate((bmi_data, additional_data))

    return bmi_data[:num_users]

# Generate BMI data for men and women
num_users_men = int(gender_counts['Male'])
num_users_women = int(gender_counts['Female'])
bmi_data_men = generate_valid_bmi(mean_bmi_men, std_dev_bmi_men, num_users_men)
bmi_data_women = generate_valid_bmi(mean_bmi_women, std_dev_bmi_women, num_users_women)

# Combine the data
bmi_data = np.concatenate((bmi_data_men, bmi_data_women))

# Plotting the combined BMI distribution
plt.figure(figsize=(10, 6))
plt.hist(bmi_data, bins=30, edgecolor='black', alpha=0.7, label='All Users', color='blue')
plt.title('User BMI Distribution')
plt.xlabel('BMI')
plt.ylabel('Frequency')
plt.grid(True)
plt.legend()
plt.show()

# Plotting the BMI distribution for men
plt.figure(figsize=(10, 6))
plt.hist(bmi_data_men, bins=30, edgecolor='black', alpha=0.7, label='Men', color='green')
plt.title('Men BMI Distribution')
plt.xlabel('BMI')
plt.ylabel('Frequency')
plt.grid(True)
plt.legend()
plt.show()

# Plotting the BMI distribution for women
plt.figure(figsize=(10, 6))
plt.hist(bmi_data_women, bins=30, edgecolor='black', alpha=0.7, label='Women', color='red')
plt.title('Women BMI Distribution')
plt.xlabel('BMI')
plt.ylabel('Frequency')
plt.grid(True)
plt.legend()
plt.show()
